In [6]:
import numbers
from datetime import timedelta, datetime
import itertools
from collections import namedtuple
import unittest
from time import sleep

In [7]:
class TimeZone:
    """a TimeZone class used to store the time zone name and offset 
    definition (in hours and minutes)"""
    def __init__(self, name, offset_hours, offset_minutes, ):
        if name is None or len(str(name).strip())==0:
            raise ValueError('Timezone name cannot be empty.')
        
        self._name = str(name).strip()
        
        if not isinstance(offset_hours, int):
            raise ValueError('Hour offset must be an integer.')
        if not isinstance(offset_minutes, int):
            raise ValueError('Minute offset must be an integer.')
            
        if abs(offset_minutes) > 59:
            raise ValueError ('Minute offset must be between -59 and 59 (inclusive)')
        
        offset = timedelta(hours=offset_hours, minutes = offset_minutes)
        if offset < timedelta(hours=-12, minutes=0) or offset > timedelta(hours=14, minutes=0):
            raise ValueError('Offset must be between -12:00 and +14:00.')
        
        self._offset_hours = offset_hours
        self._offset_minutes = offset_minutes
        self._offset = offset

    @property
    def offset(self):
        return self._offset    
   
    @property
    def name(self):
        return self._name
    
    def __eq__(self, other):
        return (isinstance(other, TimeZone) and 
                self.name == other.name and 
                self._offset == other.offset)
    
    def __repr__(self):
        return f"TimeZone(name={self.name},  offset_hours={self._offset_hours}, offset_minutes={self._offset_minutes}."
        
      

In [22]:
class Account:
    _interest_rate = 0.5
    transaction_counter = itertools.count(0)
    _transaction_codes = {"deposit":"D", 
                          "withdraw":"W",
                          "interest":"I",
                          "rejected":"X"}
    
    def __init__(self, account_number, first_name, last_name,
                 timezone = None, initial_balance = 0):
        self._account_number = account_number
        self.first_name = first_name
        self.last_name = last_name
        self._balance = Account.validate_real_number(initial_balance, min_value=0)
        if not timezone:
            timezone = TimeZone('UTC', 0,0)
        self.timezone=timezone
        
    @staticmethod
    def validate_real_number(value, min_value = None):
        if not isinstance(value, numbers.Real):
            raise ValueError('Value must be a real number.')
            
        if min_value is not None and value < min_value:
             raise ValueError(f'Value must be at least {min_value}.')
        return value
        
    @staticmethod
    def parse_confirmation_code(confirmation_code, preferred_time_zone=None):
        parts = confirmation_code.split('-')
        if len(parts) != 4:
            raise ValueError('Invalid confirmation code')
        transaction_code, account_number, raw_dt_utc, transaction_id = parts
        try:
            dt_utc = datetime.strptime(raw_dt_utc,"%Y%m%d%H%M%S" )
        except ValueError as ex:
            raise ValueError('Invalid transaction datetime.') from ex
        if not preferred_time_zone:
            preferred_time_zone = TimeZone('UTC', 0,0)
        if not isinstance(preferred_time_zone, TimeZone):
            raise ValueError("Invalid TimeZone.")
            
        df_preferred = dt_utc + preferred_time_zone.offset
        df_preferred_str = f"{df_preferred.strftime('%Y-%m-%d %H:%M:%S')} ({preferred_time_zone.name})"
        return Confirmation(account_number, transaction_code, transaction_id, dt_utc.isoformat(), df_preferred_str)
            
        
    def generate_confirmation_code(self, transaction_code):
        dt = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        return f"{transaction_code}-{self.account_number}-{dt}-{next(Account.transaction_counter)}"
    
    
    @classmethod
    def get_interest_rate(cls):
        return cls._interest_rate
    
    @classmethod
    def set_interest_rate(cls, value):
        if not isinstance(value, numbers.Real):
            raise ValueError('Interest rate must be a real number.')
        if value < 0:
            raise ValueError('Interest rate cannot be negative.')
        cls._interest_rate = value
            
    def deposit(self, value):
        value = Account.validate_real_number(value, 0.01)
        
        transaction_code = Account._transaction_codes['deposit']
        conf_code = self.generate_confirmation_code(transaction_code)
        self._balance += value
        return conf_code   
    
    def withdraw(self, value):
        value = Account.validate_real_number(value, 0.01)
        accepted = False    
        if (self.balance - value) < 0:
            transaction_code = Account._transaction_codes['rejected'] 
        else:
            accepted = True
            transaction_code = Account._transaction_codes['withdraw'] 
    
        conf_code = self.generate_confirmation_code(transaction_code)
        if accepted:
            self._balance -= value
        return conf_code   
    
    def pay_interest(self):
        interest = self.balance * Account.get_interest_rate() / 100
        conf_code = self.generate_confirmation_code(Account._transaction_codes['interest'])
        self._balance += interest
        return conf_code
        
        
    @property
    def balance(self):
        return self._balance
        
        
    @property
    def timezone(self):
        return self._timezone
    
    @timezone.setter
    def timezone(self, value):
        print("timezone setter...")
        if not isinstance(value, TimeZone):
            raise ValueError("Time zone must be a valid TimeZone object")
        self._timezone = value
        
    @property
    def account_number(self):
        return self._account_number
        
    @property
    def first_name(self):
        print('call first name property')
        return self._first_name
    
    @first_name.setter
    def first_name(self, value):
        self.validate_and_set_name('_first_name', value, "first_name")
        
    @property
    def last_name(self):
        return self._last_name
    
    @last_name.setter
    def last_name(self, value):
        self.validate_and_set_name('_last_name', value, "last_name")
        
    @property
    def full_name(self):
        return f'{self.first_name} {self.last_name}'
    
    
    def validate_and_set_name(self, attr_name, value, field_title):
        if not value or len(str(value).strip()) == 0:
            raise ValueError(f'{field_title} name cannot be empty')
        setattr(self, attr_name, value)

test suite

In [9]:
def run_tests(test_class):
    suite = unittest.TestLoader().loadTestsFromTestCase(test_class)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(suite)

разница между assert и assertEqual

In [10]:
class TestAccountExample(unittest.TestCase):
    def test_ok(self):
        assert 1==1
    def test_ok2(self):
        self.assertEqual(1,1)
    def test_not_ok(self):
        assert 1==0
    def test_not_ok2(self):
        self.assertEqual(1,0)

In [11]:
run_tests(TestAccountExample)

test_not_ok (__main__.TestAccountExample) ... FAIL
test_not_ok2 (__main__.TestAccountExample) ... FAIL
test_ok (__main__.TestAccountExample) ... ok
test_ok2 (__main__.TestAccountExample) ... ok

FAIL: test_not_ok (__main__.TestAccountExample)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-b41c0ab67ad3>", line 7, in test_not_ok
    assert 1==0
AssertionError

FAIL: test_not_ok2 (__main__.TestAccountExample)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-10-b41c0ab67ad3>", line 9, in test_not_ok2
    self.assertEqual(1,0)
AssertionError: 1 != 0

----------------------------------------------------------------------
Ran 4 tests in 0.012s

FAILED (failures=2)


setUp вызывается каждый раз

In [12]:
class TestAccountExample1(unittest.TestCase):
    def setUp(self):
        print('running setup...')
        self.x = 100
        
    def test1(self):
        sleep(0.5)
        print('running test1...')
        self.x = 200
        self.assertEqual(200, self.x)
        
    def test2(self):
        sleep(0.5)
        print('running test2...')
        self.assertEqual(100, self.x)

In [13]:
run_tests(TestAccountExample1)

test1 (__main__.TestAccountExample1) ... 

running setup...


ok
test2 (__main__.TestAccountExample1) ... 

running test1...
running setup...
running test2...


ok

----------------------------------------------------------------------
Ran 2 tests in 1.019s

OK


tearDown тоже вызывается каждый раз

In [14]:
class TestAccountExample2(unittest.TestCase):
    def setUp(self):
        print('running setup...')
        self.x = 100
        
    def tearDown(self):
        print('running teardown...') 
        
    def test1(self):
        sleep(0.5)
        print('running test1...')
        self.x = 200
        self.assertEqual(200, self.x)
        
    def test2(self):
        sleep(0.5)
        print('running test2...')
        self.assertEqual(100, self.x)

In [15]:
run_tests(TestAccountExample2)

test1 (__main__.TestAccountExample2) ... 

running setup...


ok
test2 (__main__.TestAccountExample2) ... 

running test1...
running teardown...
running setup...
running test2...
running teardown...


ok

----------------------------------------------------------------------
Ran 2 tests in 1.019s

OK


self.subTest;

change self in every test

In [20]:
class TestAccount(unittest.TestCase):
    def setUp(self):
        self.account_number = 'A100'
        self.first_name = 'FIRST'
        self.last_name = 'LAST'
        self.tz = TimeZone('ABC', -1, -30)
        self.balance = 100.00
        
    def create_account(self):
        return Account(self.account_number, self.first_name,
                       self.last_name, self.tz, self.balance)
            
    def test_timezones_not_equal(self):
        
        test_timezones = (
            TimeZone('DEF', -1, -30),
            TimeZone('ABC', -1, 0),
            TimeZone('ABC', 1, -30)
        )
        for i, test_tz in enumerate(test_timezones):
            with self.subTest(test_number=i):
                self.assertNotEqual(self.tz, test_tz)
        
    
    def test_create_account(self):
        
        
        a = self.create_account()
        self.assertEqual(self.account_number, a.account_number)
        self.assertEqual(self.first_name, a.first_name)
        self.assertEqual(self.last_name, a.last_name)
        self.assertEqual(self.first_name + ' ' + self.last_name, a.full_name)
        self.assertEqual(self.tz, a.timezone)
        self.assertEqual(self.balance, a.balance)
        
    def test_create_account_blank_first_name(self):
        
        self.first_name = ''
        with self.assertRaises(ValueError):
            self.create_account()
            
    def test_create_account_negative_balance(self):
        
        self.balance = -100.00
        
        with self.assertRaises(ValueError):
            self.create_account()
            
        

In [23]:
run_tests(TestAccount)

test_create_account (__main__.TestAccount) ... ok
test_create_account_blank_first_name (__main__.TestAccount) ... ok
test_create_account_negative_balance (__main__.TestAccount) ... ok
test_timezones_not_equal (__main__.TestAccount) ... 

timezone setter...
call first name property
call first name property


ok

----------------------------------------------------------------------
Ran 4 tests in 0.005s

OK
